In [1]:
#Import packages
import xarray as xr
import pandas as pd
import numpy as np
import datetime as dt
t
import geocat.comp
from datetime import datetime
import dask
import zarr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import netCDF4
import os
import proplot as pplt
import warnings
from numpy import meshgrid, deg2rad, gradient, cos, sin
from xarray import DataArray
from scipy import stats
from scipy.interpolate import griddata
from sklearn.preprocessing import StandardScaler as scale
warnings.filterwarnings('ignore')

In [2]:
# Connect to AWS S3 storage
fs = s3fs.S3FileSystem(anon=True)

## By downloading the master CSV file enumerating all available data stores, we can interact with the spreadsheet
## through a pandas DataFrame to search and explore for relevant data using the CMIP6 controlled vocabulary:

df = pd.read_csv("https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.csv")

In [3]:
def getData(qstring):
    df_subset = df.query(qstring)
    if df_subset.empty:
        print('data not available for '+qstring)
    else:
        for v in df_subset.zstore.values:
            zstore = v
            mapper = fs.get_mapper(zstore)
            return_ds = xr.open_zarr(mapper, consolidated=True)
    return(return_ds)

In [20]:
#Load in model data
modelNames=['CESM2','CESM2-FV2','CESM2-WACCM','CESM2-WACCM-FV2','CMCC-CM2-SR5','CanESM5',
'E3SM-1-1','E3SM-1-1-ECA','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H','MIROC-ES2L','MIROC6'
,'MPI-ESM-1-2-HAM','MPI-ESM1-2-HR','MRI-ESM2-0']

membList=["'r1i1p1f1'"]

precmeanlist=[]
cresmeanlist=[]

for m,i in zip(modelNames,membList):
    inputStr  = "activity_id=='CMIP' & table_id=='Amon' & experiment_id=='historical' &  member_id=='r1i1p1f1' & variable_id=="
    altinputStr  = "activity_id=='CMIP' & table_id=='fx' & experiment_id=='historical' &  member_id=='r1i1p1f1' & variable_id=="
    rsut_ds   = getData(inputStr+"'rsut'")
    pr_ds     = getData(inputStr+"'pr'")
    rsutcs_ds = getData(inputStr+"'rsutcs'")
    sftlf_ds = getData(altinputStr+"'sftlf'")
   
    cres= rsutcs_ds.rsutcs-rsut_ds.rsut
    prec=pr_ds.pr
    land=sftlf_ds.sftlf
    
    lats=prec.lat
    lons=prec.lon
    landlat=land.lat
    landlon=land.lon
    
    land=land.to_numpy()
    #Regrid land to a common grid
    X, Y =np.meshgrid(landlon,landlat)
    lat1=lats
    lon1=lons
    XI,YI=np.meshgrid(lon1,lat1)
    
    regridland=griddata((X.flatten(),Y.flatten()), land.flatten(), (XI,YI), method='nearest')
    regridland=xr.DataArray(regridland,coords=[lat1,lon1],dims=['lat','lon'])
    regridland=np.expand_dims(regridland,axis=0)
    
    #Select July Month Indexes
    month_idxs=cres.groupby('time.month').groups
    july_idxs=month_idxs[7]
    cres=cres.isel(time=july_idxs)
    prec=prec.isel(time=july_idxs)
    time=cres.time
    
    t1=cres.stack(z=("lat", "lon"))
    # fit scaler on training data
    norm = scale().fit(t1)
    # transform training data
    t1.values = norm.transform(t1)
    cres.values=t1.unstack()
    
    
    #Select only the SAM lat,lon range: 60-100E, 10-30N 
    regridland=regridland.sel(lon=slice(60,100),lat=slice(10,30))
    precip=prec.sel(lon=slice(60,100),lat=slice(10,30))
    precip=xr.where(regridland==0,np.nan,precip) #remove oceans, monsoon is defined as only over land 

    #Do weighted correction on precipitation
    weights=np.cos(np.deg2rad(precip.lat))
    prec_index=precip.weighted(weights).mean(dim=('lat','lon'))
    prec_index=prec_index*60*60*24 #conversion to mm/day, exluding dividing by rho and multiplying by 1000mm/m

    prec_index=prec_index.to_numpy()
    t3=prec_index
    t3=t3.reshape(-1,1)
    sc_t=scale()
    t3=sc_t.fit_transform(t3)
    prec_index=t3.mean(axis=1)
    print(prec_index)
    

AttributeError: 'numpy.ndarray' object has no attribute 'sel'

In [ ]:
def detrend(x:np.ndarray,time:np.ndarray):
        nt,nx,ny = x.shape
        xtemp = x.reshape(nt,nx*ny)
        p = np.polyfit(time, xtemp, deg=3)
        #fit=p[0]*(time[:,np.newaxis])+ p[1]
        fit = p[0]*(time[:,np.newaxis] **3)+ p[1]*(time[:,np.newaxis]**2) + p[2]*(time[:,np.newaxis]) + p[3]
        return x - fit.reshape(nt,nx,ny)

def remove_time_mean(x):
        return x - x.mean(dim='time')

def removeSC(x):
        return x.groupby('time.month').apply(remove_time_mean)

In [ ]:
cres_RMSC=removeSC(cres)

#Detrend data sets
time=cres_RMSC.time
cres_july=cres_RMSC.to_numpy()


time=time.to_numpy()

time=time.astype(int)/10**9

cres_july=detrend(cres_july,time)
cres_july=xr.DataArray(cres_july,coords=[time,lat,lon],dims=['time','lat','lon'])

#test=xr.where(cres_july<0,10,cres_july)
#test1=xr.where(test==0,10,test)
#print(test1.min())

cres_july=xr.where(np.logical_and(cres_july>-1e-14, cres_july<1e-14),0,cres_july)

#qq=xr.where(cres_july<=0,10,cres_july)
#print(qq.min())


#cres_july=cres_july.shift(time=3)

test1=cres_july[8,:,:]
test=np.mean(cres_july,axis=0)
test1.plot()

In [ ]:
#Specify model names
modelNames=['CESM2','CESM2-FV2','CESM2-WACCM','CESM2-WACCM-FV2','CMCC-CM2-SR5',
'E3SM-1-1','E3SM-1-1-ECA','FGOALS-g3','GFDL-CM4','GFDL-ESM4','GISS-E2-1-G-CC','SAM0-UNICON'] 

#Specify model names
#modelNames=['CESM2','CESM2-FV2','CESM2-WACCM','CESM2-WACCM-FV2','CMCC-CM2-SR5','E3SM-1-0',
#'E3SM-1-1','E3SM-1-1-ECA','EC-Earth3','FGOALS-f3-L','FGOALS-g3','FIO-ESM-2-0','GFDL-CM4','GFDL-ESM4',
#'GISS-E2-1-G-CC','IITM-ESM','INM-CM4-8','INM-CM5-0','IPSL-CM6A-LR','KIOST-ESM','MIROC6','MPI-ESM-1-2-HAM',
#'MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-LM','NorESM2-MM','SAM0-UNICON','TaiESM1']

In [ ]:
cresmeanfinal=cresmeanfinal[5,:,:]
cresmeanfinal.plot()

In [ ]:
files='/home/jupyter-dipti/work/ISMR_CITCZ/Data/CESM2_r1i1p1f1_historical.nc'
data=xr.open_dataset(files)
data

In [ ]:
download_path='/home/jupyter-dipti/work/ISMR_CITCZ/Data/'
inputstr='_r1i1p1f1_historical.nc'

precmeanlist=[]
cresmeanlist=[]

fig,axes=pplt.subplots([[1]],proj='eqearth')
#fig.save('SAMonsoon_Intermodel_Variability')
for i,name in enumerate(modelNames):
    #get input variables in xarray
    files=download_path+name+inputstr
    data=xr.open_dataset(files)        
    lons=data['lon']
    lats=data['lat']
    
    #Regrid land to a common grid
    X, Y =np.meshgrid(landlon,landlat)
    lat1=lats
    lon1=lons
    XI,YI=np.meshgrid(lon1,lat1)
    
    regridland=griddata((X.flatten(),Y.flatten()), land.flatten(), (XI,YI), method='nearest')
    regridland=xr.DataArray(regridland,coords=[lat1,lon1],dims=['lat','lon'])
    regridland=np.expand_dims(regridland,axis=0)
    
    cres=(data.rsdt-data.rsut)-(data.rsdt-data.rsutcs)
    
    #Done getting input variables, now get output variables - precipition
    prec=data.pr
    prec=prec*60*60*24
    prec=np.multiply(regridland,prec)
    precip=xr.where(prec==0,np.nan,prec) #remove oceans, monsoon is defined as only over land 
    
    datetimeindex=cres.indexes['time'].to_datetimeindex()
    cres['time']=datetimeindex
    precip['time']=datetimeindex

    
    #Select July index
    month_idxs=cres.groupby('time.month').groups
    july_idxs=month_idxs[7]
    test_idxs=month_idxs[7]
    cres_july=cres.isel(time=july_idxs)
    prec_july=precip.isel(time=july_idxs)
    
    #cres=cres_july.shift(time=3)

    prec_july=xr.where(prec_july<0,0,prec_july) #apply Non-negative precipitation physical constraint

    t1=cres.stack(z=("lat", "lon"))
    # fit scaler on training data
    norm = scale().fit(t1)
    # transform training data
    t1.values = norm.transform(t1)
    cres.values=t1.unstack()
    #cres=cres.shift(time=3)
    
    #Select only the SAM lat,lon range: 60-100E, 10-30N 
    prec_july=prec_july.sel(lon=slice(60,100))
    prec_july=prec_july.sel(lat=slice(10,30))

    #Do weighted correction on precipitation
    weights=np.cos(np.deg2rad(prec.lat))
    prec_index=prec_july.weighted(weights).mean(dim=('lat','lon')) 
    
    #Calculate mean of precipitation index, gives a single value for each model
    mymean=prec_index.mean()
    
    precmeanlist.append(mymean) #append list of precipitation standard deviations for each model
    #Calculate model CRE , gives a mean for each grid cell 
    cresmean=cres.mean(axis=0)
    
    cresmean=cresmean.to_numpy()

    #Regrid all models to a common grid
    X, Y =np.meshgrid(lons,lats)
    lat1=np.linspace(90,-90,360)
    lon1=np.linspace(0,360,720)
    XI,YI=np.meshgrid(lon1,lat1)
    
    regridcresmean=griddata((X.flatten(),Y.flatten()), cresmean.flatten(), (XI,YI), method='nearest')
    cresmeanlist.append(regridcresmean) #append list of cres standard deviations for each model
    
    cresmeanstack=np.stack(cresmeanlist,axis=0) #stack the lists to obtain the correct dimensions


prmean=xr.DataArray(data=precmeanlist,coords={'models':np.arange(1,len(modelNames)+1)}) 


condition=np.mean(prmean)+2*np.std(prmean) #calculate the condition to be used in regression calculation

cresmeanfinal=xr.DataArray(data=cresmeanstack,coords={'models':np.arange(1,len(modelNames)+1),'lat':lat1,'lon':lon1})

prfinal,cresmeanfinal = xr.broadcast(prmean,cresmeanfinal) #broadcast prmean array to fill array to allow regression to be executed


#define the regression function
def linear_trend(x,y): 
    mask=np.isfinite(x)&np.isfinite(y)
    if len(x[mask])==0:
        return np.nan
    else:
        pf=np.polyfit(x[mask],y[mask],1)
        return xr.DataArray(pf[0])

def ints(x,y):
    mask=np.isfinite(x)&np.isfinite(y)
    if len(x[mask])==0:
        return np.nan
    else:
        ipf=np.polyfit(x[mask],y[mask],1)
        return xr.DataArray(ipf[1])

#Apply regression function using ufunc to get regression coefficients/slopes
slopes=xr.apply_ufunc(linear_trend,
                    prfinal,cresmeanfinal,
                    vectorize=True,
                    dask='parallelized',
                    input_core_dims=[['models'],['models']],
                    )



intercepts=xr.apply_ufunc(ints,
                    prfinal,cresmeanfinal,
                    vectorize=True,
                    dask='parallelized',
                    input_core_dims=[['models'],['models']],
                    )

cresregression=slopes*condition+intercepts
con=axes.contourf(lon1,lat1,cresregression,extend='both',colorbar_kw={'label': 'Wm-2'})
axes.set_title('CRES Regression')

fig.format(coast=True)
axes.format(suptitle='South Asian Monsoon July Precipitation CRES Intermodel Variability Regression')
cbar=plt.colorbar(con)

In [ ]:
#Plotting the data
varList=['cres','cresSurf','acres','crel','crelSurf','acrel']
regressions=np.stack((cresregression,cresSurfregression,acresregression,crelregression,crelSurfregression,acrelregression))
fig,axes=pplt.subplots([[1,2,3],[4,5,6]],proj='eqearth')
for j,vname in enumerate(varList):
    con=axes[j].contourf(lon1,lat1,regressions[j],extend='both',colorbar_kw={'label': 'Wm-2'})
    axes[j].set_title(vname)
    
fig.format(coast=True)
axes.format(suptitle='South Asian Monsoon July Precipitation CRE Intermodel Variability Regression')
cbar=plt.colorbar(con)
#fig.save('SAMonsoon_Intermodel_Variability')